**Building a Chatbot**

Here, we will see how  to design and implement a LLM powered chatbot. This chatbot will be able to have a conversation and capability to remember previous interactions.

Note: In this project, this chatbot will only use the language model to have a conversation. There are several other related concepts that you may be looking for: 
* **Conversation RAG:** Enable a chatbot experience over an external source of data.
* **Agents:** Build a chatbot that can take actions.

---
Here Basic chatbot is implemented only

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [6]:
from langchain_groq import ChatGroq
model=ChatGroq(model="deepseek-r1-distill-llama-70b",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016AD4A8FCD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016AD4AB8E50>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import HumanMessage
humanMsg=HumanMessage(content="Hi, Name is Rohit and I am learning GenAI")
model.invoke([humanMsg])

AIMessage(content="<think>\n\n</think>\n\nHi Rohit! Welcome to the world of GenAI learning! It's exciting that you're diving into this field. How did you get interested in GenAI, and what specific areas or applications are you most curious about? Let me know how I can assist you on your learning journey!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 15, 'total_tokens': 78, 'completion_time': 0.229090909, 'prompt_time': 0.003680159, 'queue_time': 0.258589854, 'total_time': 0.232771068}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_492bd52206', 'finish_reason': 'stop', 'logprobs': None}, id='run-00cdc067-0297-4e43-a748-0efa13d232c6-0', usage_metadata={'input_tokens': 15, 'output_tokens': 63, 'total_tokens': 78})

Applying basic approach to see that whether the model is remembering the previous response or not.

In [9]:
from langchain_core.messages import AIMessage
model.invoke([
  humanMsg,
  AIMessage(content="Hi Rohit! Welcome to the world of GenAI learning! It's exciting that you're diving into this field. How did you get interested in GenAI, and what specific areas or applications are you most curious about? Let me know how I can assist you on your learning journey!"),
  HumanMessage(content="Hey, what is my name and what I am doing?")
])

AIMessage(content='<think>\nOkay, the user just asked, "Hey, what is my name and what I am doing?" Let me check the conversation history. In the previous message, the user introduced himself as Rohit and mentioned he\'s learning GenAI. So, I should use that information to respond.\n\nI need to make sure I address him by name to keep it personal. Also, I should acknowledge his learning journey in GenAI. Maybe I can offer further assistance to show I\'m here to help.\n\nHmm, should I add some encouragement? That might make the response more friendly and supportive. Let me phrase it in a way that invites him to ask more questions if he needs help.\n</think>\n\nYour name is Rohit, and you\'re learning about GenAI! How can I assist you further on your learning journey? 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 88, 'total_tokens': 255, 'completion_time': 0.607272727, 'prompt_time': 0.00960383, 'queue_time': 0.251616223, 'total_tim

---
**Message History** \
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and stores them in some datastore. \
Future interactions will then load those messages and pass them into the chain as part of the input.

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
  if session_id not in store:
    store[session_id]=ChatMessageHistory()
  return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [13]:
config={"configurable":{"session_id":"chat1"}}

In [16]:
response=with_message_history.invoke([
  humanMsg,
],config=config)
response

AIMessage(content="\n\nHello, Rohit! It's great to hear that you're learning about General Artificial Intelligence (GenAI). This field is rapidly evolving and offers a lot of exciting opportunities. Here are some key points to keep in mind as you progress in your learning journey:\n\n### 1. **Understanding GenAI:**\n   - **Definition**: GenAI refers to advanced AI systems that can perform a wide range of tasks, similar to human intelligence. These systems are designed to be versatile and adaptable across different domains.\n   - **Applications**: GenAI can be applied in areas like natural language processing, computer vision, robotics, and more.\n\n### 2. **Starting with the Basics:**\n   - **Machine Learning (ML)**: Begin by understanding the fundamentals of machine learning, including supervised, unsupervised, and reinforcement learning.\n   - **Deep Learning (DL)**: Dive into deep learning concepts, such as neural networks, convolutional neural networks (CNNs), and recurrent neural 

In [17]:
response.content

"\n\nHello, Rohit! It's great to hear that you're learning about General Artificial Intelligence (GenAI). This field is rapidly evolving and offers a lot of exciting opportunities. Here are some key points to keep in mind as you progress in your learning journey:\n\n### 1. **Understanding GenAI:**\n   - **Definition**: GenAI refers to advanced AI systems that can perform a wide range of tasks, similar to human intelligence. These systems are designed to be versatile and adaptable across different domains.\n   - **Applications**: GenAI can be applied in areas like natural language processing, computer vision, robotics, and more.\n\n### 2. **Starting with the Basics:**\n   - **Machine Learning (ML)**: Begin by understanding the fundamentals of machine learning, including supervised, unsupervised, and reinforcement learning.\n   - **Deep Learning (DL)**: Dive into deep learning concepts, such as neural networks, convolutional neural networks (CNNs), and recurrent neural networks (RNNs).\n

In [18]:
with_message_history.invoke([
  HumanMessage(content="What is my name?")
],config=config)

AIMessage(content='<think>\n\nYour name is Rohit.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 1660, 'total_tokens': 1669, 'completion_time': 0.032727273, 'prompt_time': 0.143628351, 'queue_time': 0.23945379500000002, 'total_time': 0.176355624}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_3de5b607fa', 'finish_reason': 'stop', 'logprobs': None}, id='run-574f8ca8-cd0a-4f90-8813-846b96fd5cca-0', usage_metadata={'input_tokens': 1660, 'output_tokens': 9, 'total_tokens': 1669})

---
Change the config->session_id changing

In [22]:
config1={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
  [HumanMessage(content="What is my name?")],
  config=config1
)
response.content

'\n\nYour name is Rohit.'

In the above scenario, session id is not change that's why model remembers the interactions.

Let's change the session id now.

In [23]:
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
  [HumanMessage(content="What is my name?")],
  config=config1
)
response.content

"<think>\n\n</think>\n\nHi! I'm DeepSeek-R1, an AI assistant independently developed by the Chinese company DeepSeek Inc. For detailed information about models and products, please refer to the official documentation."

As we can see above, by changing the session_id the model do not remember the interaction

In [24]:
response=with_message_history.invoke(
  [HumanMessage(content="Hey, my name is Rahul")],
  config=config1
)
response.content

"<think>\nAlright, the user just told me their name is Rahul. I should respond in a friendly and welcoming manner. Since they shared their name, I can use it to make the interaction more personal. I want to make sure they feel comfortable asking for help, so I'll keep the tone positive and open. Maybe I can also offer further assistance to encourage them to continue the conversation.\n</think>\n\nHey Rahul! How can I assist you today?"

In [26]:
response=with_message_history.invoke(
  [HumanMessage(content="What is my name?")],
  config=config1
)
response.content

'\n\nYou told me your name is Rahul. Let me know how I can assist you!'

---
---
**Prompt Templates**\
Prompt Templates help to turn raw user info into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM.

In [38]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
  [
    ("system","You are a helpful assistant. Answer the questions to be best of your ability"),
    MessagesPlaceholder(variable_name="messages")
  ]
)

chain=prompt|model

In [39]:
chain.invoke(
  {"messages":[HumanMessage(content="Hi my name is Rohit")]}
)

AIMessage(content='<think>\nOkay, so I just saw this message where someone named Rohit introduced himself. I need to figure out how to respond appropriately. Since I\'m supposed to be a helpful assistant, I should make sure my reply is friendly and open. \n\nFirst, I should acknowledge his greeting. Maybe start with a hello or hi. Then, I can ask how I can assist him today. It\'s important to keep it simple and welcoming so he feels comfortable asking for help.\n\nI should avoid making it too long; just a brief response. Also, I should use a friendly tone without any formal language. That way, it feels more approachable.\n\nWait, in the example response, they used "Hello Rohit! How can I assist you today?" That\'s pretty straightforward. Maybe I can use that as a model but put it in my own words. Or maybe just stick with that since it\'s concise and effective.\n\nI don\'t want to overcomplicate it. The main goal is to respond in a way that invites him to ask for help. So, keeping it sh

In [40]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [41]:
config3={"configurable":{"session_id":"chat3"}}

In [42]:
response=with_message_history.invoke(
  [HumanMessage(content="Hi, my name is Rohit")],
  config=config3
)

In [43]:
response.content

'\n\nHi Rohit! Welcome. How can I assist you today?'

Adding some complexity in prompt template

In [44]:
prompt=ChatPromptTemplate.from_messages(
  [
    ("system","You are a helpful assistant. Answer the questions to be best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
  ]
)

chain=prompt|model

In [47]:
response=chain.invoke(
  {
    "messages":[HumanMessage(content="Hi my name is Rohit")],
    "language":"Hindi"
  }
)

In [48]:
response.content

'<think>\nOkay, the user greeted me with "Hi my name is Rohit." I should respond in a friendly and welcoming manner. Since they mentioned their name, I\'ll include that to make it personal. I should also offer my assistance to show I\'m ready to help with whatever they need. Keeping it simple and warm should make them feel comfortable to ask questions.\n</think>\n\nनमस्ते Rohit! मैं आपकी मदद के लिए यहाँ हूँ। आप क्या जानना चाहते हैं या मैं आपकी कैसे सहायता कर सकता हूँ?'

Now wrap this more complicated chain in a Message History class. Now because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [49]:
with_message_history=RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_messages_key="messages"
)

In [51]:
config4={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
  {
    "messages":[HumanMessage(content="Hi I am Rohit Gupta")],
    "language":"Hindi"
  },
  config=config4

)
response.content

"<think>\nAlright, the user greeted me in Hindi. I should respond warmly in the same language.\n\nThey introduced themselves as Rohit Gupta. I'll acknowledge that and offer my help.\n\nI need to keep it friendly and open-ended so they feel comfortable asking for assistance.\n</think>\n\nनमस्ते रोहित गुप्ता जी! मैं आपकी मदद के लिए तैयार हूँ। कृपया अपना प्रश्न बताएं या बताएं कि मैं आपकी कैसे सहायता कर सकता हूँ।"

In [52]:
response=with_message_history.invoke(
  {
    "messages":[HumanMessage(content="Hey, what is my name?")],
    "language":"Hindi"
  },
  config=config4
)

response.content

'<think>\nAlright, the user just said "Hey, what is my name?" in English. \n\nLooking back at the history, the user introduced himself as Rohit Gupta earlier.\n\nI should respond in Hindi as per the initial instruction.\n\nI need to confirm his name politely.\n\nSo, I\'ll say his name is Rohit Gupta and offer further assistance.\n</think>\n\nआपका नाम रोहित गुप्ता है, सही है ना? कृपया बताएं कि मैं आपकी और कैसे मदद कर सकता हूँ!'

---
**Managing the Conversation History**

If the chatbot conversation history is unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM.\
Therefore, it is important to add a step that limits the size of the messages you are passing in.

`trim_messages` : helper to redunce the number of messages we are sending to the model. \
The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [60]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
  max_tokens=50,
  strategy="last",
  token_counter=model,
  include_system=True,
  allow_partial=False,
  start_on="human"
)
messages=[
  SystemMessage(content="You are a good assistant"),
  HumanMessage(content="Hi, I am Rohit"),
  AIMessage(content="Hi!"),
  HumanMessage(content="I like vanilla ice cream"),
  AIMessage(content="nice"),
  HumanMessage(content="what is 2+2?"),
  AIMessage(content="4"),
  HumanMessage(content="thanks"),
  AIMessage(content="No problem!"),
  HumanMessage(content="Have fun.."),
  AIMessage(content='woah!!')
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Have fun..', additional_kwargs={}, response_metadata={}),
 AIMessage(content='woah!!', additional_kwargs={}, response_metadata={})]

Creating a chain with trimmer

In [62]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
  RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
  | prompt
  | model
)

response=chain.invoke(
  {
    "messages":messages + [HumanMessage(content="What icecream do I like")],
    "language":"English"
  }
)

response.content

'<think>\nAlright, the user just asked, "What ice cream do I like." Hmm, I don\'t have access to personal data, so I can\'t know their preferences. I should let them know that. Maybe I can offer some popular options to help them decide. Let me list a few common flavors like vanilla, chocolate, strawberry, and maybe some more unique ones like cookies and cream or mint chocolate chip. That way, they can think about which one they might like. I should keep it friendly and open-ended, encouraging them to share if they want. Yeah, that sounds good.\n</think>\n\nI don\'t have access to personal data, so I don\'t know your preferences, but I can suggest some popular ice cream flavors if you\'d like! For example, vanilla, chocolate, strawberry, cookies and cream, or mint chocolate chip. Let me know if you\'d like more suggestions or help with anything else! 😊'

In above, the trimmer removed the message related to ice cream bcz of maximum tokens, That is why the model is unable to repond correctly.

In [64]:
response=chain.invoke(
  {
    "messages":messages + [HumanMessage(content="What math problem did I ask?")],
    "language":"English"
  }
)

response.content

'<think>\nAlright, the user is asking, "What math problem did I ask?" Let me check the history. \n\nLooking back, the user previously asked, "what is 2+2?" and I responded with "4". \n\nSo, they\'re now inquiring about the specific math problem they had asked before. \n\nI should confirm the problem and the answer to make sure they\'re on the same page. \n\nMaybe they\'re trying to recall or verify the question for their records or further use. \n\nI\'ll restate the problem and the solution clearly to provide the information they need.\n</think>\n\nYou asked, "What is 2+2?" and the answer was **4**. 😊'

Wrap up in Message history

In [65]:
with_message_history=RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_messages_key="messages"
)

config={"configurable":{"session_id":"chat5"}}

In [67]:
response=with_message_history.invoke(
 { 
   "messages":messages+[HumanMessage(content="what is my name?")],
   "language":"English"
 },
  config=config
)
response.content

'<think>\nAlright, the user asked "what is my name?" in their last message. I need to figure out how to respond appropriately. \n\nLooking at the conversation history, I see that I\'ve been helpful so far. They asked for 2+2 and I gave the correct answer, then they thanked me, and I responded politely. Now they\'re asking for their name.\n\nI don\'t have access to personal information about the user unless they\'ve shared it before. Since this is the start of our interaction, I can assume I don\'t know their name. \n\nI should let them know that I can\'t access personal information. It\'s important to respect their privacy and be transparent about my limitations. \n\nMaybe I can offer to help with something else to keep the conversation going. That way, they know I\'m still here to assist them in other ways.\n</think>\n\nI don\'t have access to personal information, including your name. If you\'d like, you can share it with me, and I\'ll do my best to assist you! 😊'

In [68]:
response=with_message_history.invoke(
 { 
   "messages":messages+[HumanMessage(content="what math problem did I ask?")],
   "language":"English"
 },
  config=config
)
response.content

'<think>\nOkay, so the user just asked, "what math problem did I ask?" Hmm, let me think. Looking back at the conversation history, the user started by asking "what is 2+2?" and I replied with "4". Then the user said "thanks" and I responded with "No problem!" After that, the user wrote "Have fun.." and I reacted with "woah!!". Now, the user is asking about the math problem they asked. \n\nWait, so the user might be a bit confused or maybe they\'re testing if I can recall our previous conversation. I should check the history to make sure. Yeah, the math problem was indeed 2+2. Maybe the user is verifying if I remember correctly or if they want to confirm something. \n\nI should respond clearly, stating that they asked for 2+2 and that the answer was 4. It\'s important to be precise and helpful. I don\'t see any other math problems in our conversation, so it\'s safe to assume that\'s the one they\'re referring to. I should also keep the tone friendly and open, inviting them to ask more 